In [3]:
import json
from pathlib import Path
import ujson
from dutchanalyzer.config import *
from dutchanalyzer.utils import *
from dutchanalyzer.json_utils import *
from pathlib import Path
from dotenv import load_dotenv
import datetime
import re
from pprint import pprint
import ast
from tqdm import tqdm
import pandas as pd
import numpy as np

In [4]:
NNF_file = Path(WIKT_CLEANING_DIR, 'nl','NNF.jsonl')
NEF_file = Path(WIKT_CLEANING_DIR, 'nl','NEF.jsonl')
EEF_file = Path(WIKT_CLEANING_DIR, 'en','EEF.jsonl')
ENF_file = Path(WIKT_CLEANING_DIR, 'en','ENF.jsonl')
NEF_definitions = Path(WIKT_CLEANING_DIR, 'nl','NEF_definitions.jsonl')
NNF_definitions = Path(WIKT_CLEANING_DIR, 'nl','NNF_definitions.jsonl')
ENF_definitions = Path(WIKT_CLEANING_DIR, 'en','ENF_definitions.jsonl')
EEF_definitions = Path(WIKT_CLEANING_DIR, 'en','EEF_definitions.jsonl')

In [5]:
today = datetime.date.today().__format__("%d-%m-%y")
current_save_folder = Path(INTERIM_DATA_DIR, 'cleaning', 'wikt', str(today))

## Flatten definitions

In [7]:
def filter_obj(obj):
    if 'translations' in obj:
        translations = []
        for t in obj['translations']:
            lang_code = t.get('lang_code')
            if lang_code == 'en' or lang_code == 'nl':
                translations.append(t)
        obj['translations'] = translations

In [73]:
def reformat_translations(translations_list: list):
    # formatted as returning en translations, nl translations and other translations if they exist
    en_translations = {}
    nl_translations = {}
    other_translations = {}
    accepted_tags = ['word', 'sense', 'sense_index', 'lang', 'lang_code', 'roman', 'tags', 'code', 'note', 'english', 'translation', 'raw_tags', 'taxonomic', 'topics', 'alt']
    for translation in translations_list:
        tags = translation.get('tags')
        note = translation.get('note')
        english = translation.get('english')
        alt = translation.get('alt')
        extra_translation_keys = False
        if translation.get('lang_code') == 'en' or translation.get('code') == 'en':
            en_translations[len(en_translations)] = { "word" : translation.get('word'), "sense": translation.get('sense')}
            if translation.get('tags'):
                 en_translations[len(en_translations) - 1]['tags'] = translation.get('tags')
            if alt:
                en_translations[len(en_translations) - 1]['alt'] = alt
            for key in translation.keys():
                if key not in accepted_tags:
                    print("word : ", translation.get('word'), " other key: ", key)
                    extra_translation_keys = True
                    break
        elif translation.get('lang_code') == 'nl' or translation.get('code') == 'nl':
            nl_translations[len(nl_translations)] = { "word" : translation.get('word'), "sense": translation.get('sense')}
            if translation.get('tags'):
                 nl_translations[len( nl_translations)  - 1]['tags'] = translation.get('tags')
            if alt:
                 nl_translations[len( nl_translations)  - 1]['alt'] = alt   
            for key in translation.keys():
                if key not in accepted_tags:
                    print("word : ", translation.get('word'), " other key: ", key)
                    extra_translation_keys = True
                    break
        else:
            other_translations[len(other_translations)] = { "word" : translation.get('word'), "sense": translation.get('sense')}
            if translation.get('tags'):
                other_translations[len(other_translations) - 1]['tags'] = translation.get('tags')
            if alt:
                other_translations[len(other_translations) - 1]['alt'] = alt
            for key in translation.keys():
                if key not in accepted_tags:
                    print("word : ", translation.get('word'), " other key: ", key)
                    extra_translation_keys = True
                    break
    all_translations = {'en' : en_translations, 'nl': nl_translations, 'other': other_translations}
    return all_translations, extra_translation_keys

In [46]:
NNF_defs_list = []

In [ ]:
NNF_defs_list = []
with open(NNF_definitions, 'r', encoding='utf-8') as f:
    for i, line in tqdm(enumerate(f)):
        
        loaded = json.loads(line)
        filter_obj(loaded)
        new_row = {"word": loaded.get("word"), "pos": loaded.get("pos"), "lang_code": loaded.get("lang_code"), "senses" : loaded.get("senses"), "wl_code":loaded.get("wl_code")}
        if loaded.get('translations'):
            translations, extra_translation_keys = reformat_translations(loaded.get('translations'))
            if translations.get('en') != {}:
                new_row['en_translations'] = translations['en']
            if translations.get('nl') != {}:
                new_row['nl_translations'] = translations['nl']
            if translations.get('other') != {}:
                new_row['other_translations'] = translations['other']
            if extra_translation_keys == True:
                print("broke on ", i)
                display(translations)
                display(loaded)
                break
        NNF_defs_list.append(new_row)

611444it [00:07, 80238.28it/s] 


In [ ]:
NNF_df = pd.DataFrame(NNF_defs_list)
NNF_df

,word,pos,lang_code,senses,wl_code,en_translations,nl_translations
0,ja,adv,nl,"{'0': {'glosses': ['duidt bevestiging, instemm...",NNF,"{0: {'word': 'yes', 'sense': 'duidt bevestigin...",NaN
1,ja,intj,nl,"{'0': {'glosses': ['kreet van opwinding']}, '1...",NNF,"{0: {'word': 'yes', 'sense': 'kreet van opwind...",NaN
2,ja,noun,nl,{'0': {'glosses': ['bevestigend of instemmend ...,NNF,"{0: {'word': 'yes', 'sense': 'bevestigend of i...",NaN
3,neen,intj,nl,{'0': {'glosses': ['ontkenning van de gestelde...,NNF,"{0: {'word': 'no', 'sense': 'nee'}}",NaN
4,een,article,nl,{'0': {'glosses': ['onbepaald lidwoord dat in ...,NNF,NaN,NaN
...,...,...,...,...,...,...,...
611439,mottigaardje,noun,nl,{'0': {'glosses': ['verkleinwoord enkelvoud va...,NNF,NaN,NaN
611440,mottigaards,noun,nl,{'0': {'glosses': ['meervoud van het zelfstand...,NNF,NaN,NaN
611441,bankpootjes,noun,nl,{'0': {'glosses': ['verkleinwoord meervoud van...,NNF,NaN,NaN
611442,bankpootje,noun,nl,{'0': {'glosses': ['verkleinwoord enkelvoud va...,NNF,NaN,NaN


In [50]:
NNF_df.to_csv(Path(current_save_folder, 'NNF_definitions_df.csv'))

In [54]:
NEF_defs_list = []
with open(NEF_definitions, 'r', encoding='utf-8') as f:
    for i, line in tqdm(enumerate(f)):
        
        loaded = json.loads(line)
        filter_obj(loaded)
        lang_code = loaded.get("lang_code")
        if not lang_code:
            lang_code = loaded.get("code")
        new_row = {"word": loaded.get("word"), "pos": loaded.get("pos"), "lang_code": lang_code, "senses" : loaded.get("senses"), "wl_code":loaded.get("wl_code")}
        if loaded.get('translations'):
            translations, extra_translation_keys = reformat_translations(loaded.get('translations'))
            if translations.get('en') != {}:
                new_row['en_translations'] = translations['en']
            if translations.get('nl') != {}:
                new_row['nl_translations'] = translations['nl']
            if translations.get('other') != {}:
                new_row['other_translations'] = translations['other']
            if extra_translation_keys == True:
                print("broke on ", i)
                display(translations)
                display(loaded)
                break
        NEF_defs_list.append(new_row)

17441it [00:00, 237574.84it/s]


In [52]:
NEF_df = pd.DataFrame(NEF_defs_list)
NEF_df.to_csv(Path(current_save_folder, 'NEF_definitions_df.csv'))

In [74]:
EEF_defs_list = []
with open(EEF_definitions, 'r', encoding='utf-8') as f:
    for i, line in tqdm(enumerate(f)):
        
        loaded = json.loads(line)
        filter_obj(loaded)
        if not lang_code:
            lang_code = loaded.get("code")
        new_row = {"word": loaded.get("word"), "pos": loaded.get("pos"), "lang_code": lang_code, "senses" : loaded.get("senses"), "wl_code":loaded.get("wl_code")}
        if loaded.get('translations'):
            translations, extra_translation_keys = reformat_translations(loaded.get('translations'))
            if translations.get('en') != {}:
                new_row['en_translations'] = translations['en']
            if translations.get('nl') != {}:
                new_row['nl_translations'] = translations['nl']
            if translations.get('other') != {}:
                new_row['other_translations'] = translations['other']
            if extra_translation_keys == True:
                print("broke on ", i)
                display(translations)
                display(loaded)
                break
        EEF_defs_list.append(new_row)

1417383it [00:14, 101033.52it/s]


In [75]:
EEF_df = pd.DataFrame(EEF_defs_list)
EEF_df.to_csv(Path(current_save_folder, 'EEF_definitions_df.csv'))

In [85]:
ENF_defs_list = []
with open(ENF_definitions, 'r', encoding='utf-8') as f:
    for i, line in tqdm(enumerate(f)):
        
        loaded = json.loads(line)
        filter_obj(loaded)
        if not lang_code:
            lang_code = loaded.get("code")
        new_row = {"word": loaded.get("word"), "pos": loaded.get("pos"), "lang_code": lang_code, "senses" : loaded.get("senses"), "wl_code":loaded.get("wl_code")}
        if loaded.get('translations'):
            translations, extra_translation_keys = reformat_translations(loaded.get('translations'))
            if translations.get('en') != {}:
                new_row['en_translations'] = translations['en']
            if translations.get('nl') != {}:
                new_row['nl_translations'] = translations['nl']
            if translations.get('other') != {}:
                new_row['other_translations'] = translations['other']
            if extra_translation_keys == True:
                print("broke on ", i)
                display(translations)
                display(loaded)
                break
        ENF_defs_list.append(new_row)

140758it [00:01, 120765.18it/s]


In [86]:
ENF_df = pd.DataFrame(ENF_defs_list)
ENF_df.to_csv(Path(current_save_folder, 'ENF_definitions_df.csv'))

In [76]:
EEF_df = EEF_df[EEF_df['pos'] != 'character']

In [77]:
EEF_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 1417326 entries, 0 to 1417382
Data columns (total 6 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   word             1417326 non-null  object
 1   pos              1417326 non-null  object
 2   lang_code        1417326 non-null  object
 3   senses           1417326 non-null  object
 4   wl_code          1417326 non-null  object
 5   nl_translations  42889 non-null    object
dtypes: object(6)
memory usage: 75.7+ MB


In [79]:
both_lang_words = EEF_df.merge(NNF_df, on=['word', 'pos'], how='inner')

In [82]:
both_lang_words = both_lang_words[both_lang_words['pos'] != 'name']

In [83]:
both_lang_words

,word,pos,lang_code_x,senses_x,wl_code_x,nl_translations_x,lang_code_y,senses_y,wl_code_y,en_translations,nl_translations_y
0,thesaurus,noun,en,{'0': {'glosses': ['A publication that provide...,EEF,"{0: {'word': 'thesaurus', 'sense': 'book of sy...",nl,{'0': {'glosses': ['een systematisch georganis...,NNF,NaN,NaN
1,gratis,adj,en,{'0': {'glosses': ['Free: without charge.']}},EEF,NaN,nl,{'0': {'glosses': ['wat zonder betaling verkre...,NNF,"{0: {'word': 'free', 'sense': 'wat zonder beta...",NaN
2,word,verb,en,{'0': {'glosses': ['To say or write (something...,EEF,"{0: {'word': 'verwoorden', 'sense': 'to say or...",nl,{'0': {'glosses': ['eerste persoon enkelvoud t...,NNF,NaN,NaN
3,word,verb,en,{'0': {'glosses': ['Alternative form of worth ...,EEF,NaN,nl,{'0': {'glosses': ['eerste persoon enkelvoud t...,NNF,NaN,NaN
4,pond,noun,en,{'0': {'glosses': ['An inland body of standing...,EEF,"{0: {'word': 'vijver', 'sense': 'small lake', ...",nl,{'0': {'glosses': ['naam voor verschillende mu...,NNF,"{0: {'word': 'pound', 'sense': 'naam voor vers...",NaN
...,...,...,...,...,...,...,...,...,...,...,...
26185,galoppades,noun,en,"{'0': {'glosses': ['plural of galoppade.'], 'f...",EEF,NaN,nl,{'0': {'glosses': ['meervoud van het zelfstand...,NNF,NaN,NaN
26186,geelbek,noun,en,{'0': {'glosses': ['Alternative form of geelbe...,EEF,NaN,nl,"{'0': {'glosses': ['dier met een gele bek']}, ...",NNF,"{0: {'word': 'African weakfish', 'sense': 'vis...",NaN
26191,longread,noun,en,{'0': {'glosses': ['A piece of (online) long-f...,EEF,NaN,nl,{'0': {'glosses': ['lang achtergrond artikel i...,NNF,NaN,NaN
26192,longreads,noun,en,"{'0': {'glosses': ['plural of longread'], 'for...",EEF,NaN,nl,{'0': {'glosses': ['meervoud van het zelfstand...,NNF,NaN,NaN


In [84]:
both_lang_words.sort_values(by='word')

,word,pos,lang_code_x,senses_x,wl_code_x,nl_translations_x,lang_code_y,senses_y,wl_code_y,en_translations,nl_translations_y
5891,'m,pron,en,{'0': {'glosses': ['Alternative form of 'em.']}},EEF,NaN,nl,{'0': {'glosses': ['clitische voorwerpsvorm vo...,NNF,NaN,NaN
11918,'t,pron,en,"{'0': {'glosses': ['Contraction of it.']}, '1'...",EEF,NaN,nl,{'0': {'glosses': ['clitische vorm van het als...,NNF,NaN,NaN
11919,'t,pron,en,"{'0': {'glosses': ['Contraction of it.']}, '1'...",EEF,NaN,nl,{'0': {'glosses': ['clitische vorm van het']}},NNF,NaN,NaN
5114,-a,suffix,en,"{'0': {'glosses': ['plural of -um'], 'forms': ...",EEF,NaN,nl,{'0': {'glosses': ['vormt een zelfstandig naam...,NNF,NaN,NaN
5115,-a,suffix,en,"{'0': {'glosses': ['plural of -on'], 'forms': ...",EEF,NaN,nl,{'0': {'glosses': ['vormt een zelfstandig naam...,NNF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4601,über,adv,en,{'0': {'glosses': ['Alternative form of uber.']}},EEF,NaN,nl,"{'0': {'glosses': ['erg veel, zeer, enorm']}}",NNF,NaN,NaN
20622,über-,prefix,en,{'0': {'glosses': ['Alternative form of uber-....,EEF,NaN,nl,"{'0': {'glosses': ['in hoge mate, heel erg']},...",NNF,NaN,NaN
26117,übercool,adj,en,{'0': {'glosses': ['Alternative form of uberco...,EEF,NaN,nl,"{'0': {'glosses': ['meer dan cool, supergaaf']}}",NNF,NaN,NaN
5415,übermensch,noun,en,{'0': {'glosses': ['An overman (“higher” man) ...,EEF,NaN,nl,{'0': {'glosses': ['persoon met aangeboren eig...,NNF,NaN,NaN


In [87]:
both_english_defs_df = EEF_df.merge(ENF_df, on=['word', 'pos'], how='inner')

In [90]:
both_english_defs_df[both_english_defs_df['pos'] != 'name']

,word,pos,lang_code_x,senses_x,wl_code_x,nl_translations,lang_code_y,senses_y,wl_code_y
0,gratis,adj,en,{'0': {'glosses': ['Free: without charge.']}},EEF,NaN,en,"{'0': {'glosses': ['free, without charge']}}",ENF
1,word,verb,en,{'0': {'glosses': ['To say or write (something...,EEF,"{0: {'word': 'verwoorden', 'sense': 'to say or...",en,"{'0': {'glosses': ['inflection of worden:', 'f...",ENF
2,word,verb,en,{'0': {'glosses': ['Alternative form of worth ...,EEF,NaN,en,"{'0': {'glosses': ['inflection of worden:', 'f...",ENF
3,pond,noun,en,{'0': {'glosses': ['An inland body of standing...,EEF,"{0: {'word': 'vijver', 'sense': 'small lake', ...",en,"{'0': {'glosses': ['unit of mass, often broadl...",ENF
4,pies,noun,en,"{'0': {'glosses': ['plural of pie'], 'forms': ...",EEF,NaN,en,{'0': {'glosses': ['alternative form of pis; p...,ENF
...,...,...,...,...,...,...,...,...,...
11352,Madame Jeanette,noun,en,{'0': {'glosses': ['A chili pepper (Capsicum c...,EEF,"{0: {'word': 'madame-jeanette', 'sense': 'chil...",en,{'0': {'glosses': ['alternative form of madame...,ENF
11353,labaria,noun,en,{'0': {'glosses': ['The fer-de-lance.']}},EEF,NaN,en,{'0': {'glosses': ['synonym of gewone lanspunt...,ENF
11354,dirty chai,noun,en,{'0': {'glosses': ['A chai latte with a shot o...,EEF,"{0: {'word': 'dirty chai', 'sense': 'chai latt...",en,{'0': {'glosses': ['dirty chai (chai latte wit...,ENF
11355,horribile auditu,adv,en,{'0': {'glosses': ['Horrible to hear.']}},EEF,"{0: {'word': 'horribile auditu', 'sense': 'hor...",en,{'0': {'glosses': ['horribile auditu (horrible...,ENF


In [91]:
def extract_single_definition(sense):
    sense = dict(sense)
    if len(sense) == 1:
        if sense['0'].get('glosses'):
            if len(sense['0']['glosses']) == 1:
                definition = sense['0']['glosses'][0]
                return definition
            else:
                return f"multi_gloss: {len(sense['0']['glosses'])}"
        else:
            return "no_gloss"
    else:
        return f"multi_sense: {len(sense)}"

In [92]:
EEF_df['definition'] = EEF_df['senses'].apply(lambda x: extract_single_definition(x))

C:\Users\elise\AppData\Local\Temp\ipykernel_1536\3680785905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EEF_df['definition'] = EEF_df['senses'].apply(lambda x: extract_single_definition(x))


In [95]:
EEF_df_path = Path(current_save_folder, 'EEF_definitions_df.csv')
EEF_df.to_csv(EEF_df_path)

In [94]:
ENF_df['definition'] = ENF_df['senses'].apply(lambda x: extract_single_definition(x))

In [97]:
ENF_df_path = Path(current_save_folder, 'ENF_definitions_df.csv')
ENF_df.to_csv(ENF_df_path)

In [102]:
NEF_df['definition'] = NEF_df['senses'].apply(lambda x: extract_single_definition(x))

In [103]:
NNF_df['definition'] = NNF_df['senses'].apply(lambda x: extract_single_definition(x))

In [105]:
equal_defs_enf = ENF_df[ENF_df['word'] == ENF_df['definition']]

In [107]:
NEF_df = NEF_df[NEF_df['pos'] != 'name']

In [109]:
NEF_df.head(10)

,word,pos,lang_code,senses,wl_code,en_translations,definition
0,busybody,noun,en,{'0': {'glosses': ['bemoeial; iemand die zich ...,NEF,NaN,bemoeial; iemand die zich bemoeit met zaken va...
1,interfere,verb,en,{'0': {'glosses': ['bemoeien']}},NEF,NaN,bemoeien
2,IPA,noun,en,{'0': {'glosses': ['biersoort gebrouwen uit li...,NEF,NaN,biersoort gebrouwen uit lichte mout waaraan ex...
7,French,noun,en,"{'0': {'glosses': ['Frans zn']}, '1': {'glosse...",NEF,NaN,multi_sense: 3
8,French,adj,en,{'0': {'glosses': ['Frans bn']}},NEF,NaN,Frans bn
9,English,noun,en,"{'0': {'glosses': ['Engels']}, '1': {'glosses'...",NEF,NaN,multi_sense: 2
10,English,adj,en,{'0': {'glosses': ['Engels']}},NEF,NaN,Engels
11,German,noun,en,{'0': {'glosses': ['Duitser; inwoner van Duits...,NEF,NaN,multi_sense: 3
12,German,adj,en,{'0': {'glosses': ['Duits; oorspronkelijk uit ...,NEF,NaN,Duits; oorspronkelijk uit Duitsland of uit het...
13,Hebrew,noun,en,{'0': {'glosses': ['Hebreeuws']}},NEF,NaN,Hebreeuws


In [110]:
NEF_df_exploded = NEF_df.explode('senses')

In [112]:
NEF_df_exploded = NEF_df.explode('senses', ignore_index=True)

In [113]:
NEF_df_exploded

,word,pos,lang_code,senses,wl_code,en_translations,definition
0,busybody,noun,en,0,NEF,NaN,bemoeial; iemand die zich bemoeit met zaken va...
1,interfere,verb,en,0,NEF,NaN,bemoeien
2,IPA,noun,en,0,NEF,NaN,biersoort gebrouwen uit lichte mout waaraan ex...
3,French,noun,en,0,NEF,NaN,multi_sense: 3
4,French,noun,en,1,NEF,NaN,multi_sense: 3
...,...,...,...,...,...,...,...
24907,decay,noun,en,0,NEF,NaN,multi_sense: 2
24908,decay,noun,en,1,NEF,NaN,multi_sense: 2
24909,decay,verb,en,0,NEF,NaN,multi_sense: 3
24910,decay,verb,en,1,NEF,NaN,multi_sense: 3


In [106]:
equal_defs_nef = NEF_df[NEF_df['word'] == NEF_df['definition']]

In [72]:
lang_codes_to_keep = ['nl', 'en', 'simple', 'ang', 'dum', 'nds', 'odt', 'nds-nl', 'enm']

In [78]:
print(len(one_gloss_defs))
print(len(multi_gloss_defs))

2165735
26278


In [80]:
display(one_gloss_defs[0:100])

[{'word': 'woordenboek',
  'pos': 'noun',
  'definition': 'dictionary',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'gratis',
  'pos': 'adj',
  'definition': 'free, without charge',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'gratuit',
  'pos': 'adj',
  'definition': 'gratuitous, not obliged to',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'pies',
  'pos': 'noun',
  'definition': 'alternative form of pis; pee, piss',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'A',
  'pos': 'character',
  'definition': 'The first letter of the Dutch alphabet, written in the Latin script.',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'raven',
  'pos': 'verb',
  'definition': 'to (hold a) rave, to party wildly',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'raven',
  'pos': 'noun',
  'definition': 'obsolete form of raaf',
  'wl_code': 'ENF',
  'lang_code': 'nl'},
 {'word': 'raven',
  'pos': 'noun',
  'definition': 'plural of raaf',
  'wl_code': 'ENF',
  'la

In [34]:
one_gloss_defs_df = pd.DataFrame(one_gloss_defs)

In [ ]:
display(one_gloss_defs_df[~one_gloss_defs_df['translations'].isna()])

,word,pos,definition,wl_code,lang_code,translations
124707,event,noun,gebeurtenis,NEF,en,"[{'lang_code': 'is', 'lang': 'IJslands', 'word..."
133321,tonic,noun,een koolzuurhoudende frisdrank met als kenmerk...,NEF,en,"[{'lang_code': 'en', 'lang': 'Engels', 'word':..."
136657,LOL,intj,hier moet ik hard om lachen,NEF,en,"[{'lang_code': 'fr', 'lang': 'Frans', 'word': ..."
139710,ja,adv,"duidt bevestiging, instemming, toestemming, in...",NNF,nl,"[{'lang_code': 'af', 'lang': 'Afrikaans', 'wor..."
139711,ja,intj,nou ja: kijk zo simpel is het! dat je dat niet...,NNF,nl,"[{'lang_code': 'de', 'lang': 'Duits', 'word': ..."
...,...,...,...,...,...,...
2164044,Tongeren-Borgloon,name,"A municipality of Limburg, Flanders, Belgium.",EEF,en,"[{'lang': 'Dutch', 'code': 'nl', 'lang_code': ..."
2164129,Evere,name,"A municipality of Brussels, Belgium.",EEF,en,"[{'lang': 'Dutch', 'code': 'nl', 'lang_code': ..."
2164130,Aarschot,name,"A city in Aerschot, Flanders, Belgium.",EEF,en,"[{'lang': 'Dutch', 'code': 'nl', 'lang_code': ..."
2164362,Lake Flevo,name,"An ancient lake in Germania, Europe.",EEF,en,"[{'lang': 'Dutch', 'code': 'nl', 'lang_code': ..."


In [38]:
dups = one_gloss_defs_df.duplicated(subset=['word'])

In [42]:
one_gloss_defs_df['duplicated'] = one_gloss_defs_df.duplicated(subset='word')

In [ ]:
one_gloss_defs_df[one_gloss_defs_df['duplicated'] == True]
one

,word,pos,definition,wl_code,lang_code,translations,duplicated
6,raven,noun,obsolete form of raaf,ENF,nl,NaN,True
7,raven,noun,plural of raaf,ENF,nl,NaN,True
14,name,noun,obsolete form of naam,ENF,nl,NaN,True
29,week,adj,"weak, gentle, weakhearted",ENF,nl,NaN,True
30,week,verb,singular past indicative of wijken,ENF,nl,NaN,True
...,...,...,...,...,...,...,...
2165608,pyridylamine,noun,Aminopyridine: an amine derived from pyridine,EEF,en,NaN,True
2165649,0/10,adj,"Bad, horrible, not good.",EEF,en,NaN,True
2165654,fcking,noun,Censored spelling of fucking.,EEF,en,NaN,True
2165687,jorks,verb,third-person singular simple present indicativ...,EEF,en,NaN,True


In [ ]:
one_gloss_defs_df['def_lang'] = one_gloss_defs_df['wl_code'].apply(lambda x: 'en' if x[0] == "E" else "nl")

In [54]:
one_gloss_defs_df.to_csv(Path(current_save_folder, 'one_gloss_defs.csv'))

In [51]:
one_gloss_defs_df.rename(columns={'def_code':'def_lang'}, inplace=True)

In [52]:
one_gloss_defs_df = one_gloss_defs_df.loc[:, ['word', 'pos', 'lang_code', 'def_lang', 'definition', 'translations', 'wl_code', 'duplicated']]

In [53]:
one_gloss_defs_df

,word,pos,lang_code,def_lang,definition,translations,wl_code,duplicated
0,woordenboek,noun,nl,en,dictionary,NaN,ENF,False
1,gratis,adj,nl,en,"free, without charge",NaN,ENF,False
2,gratuit,adj,nl,en,"gratuitous, not obliged to",NaN,ENF,False
3,pies,noun,nl,en,"alternative form of pis; pee, piss",NaN,ENF,False
4,A,character,nl,en,"The first letter of the Dutch alphabet, writte...",NaN,ENF,False
...,...,...,...,...,...,...,...,...
2165730,machine taper,noun,en,en,A system for securing cutting tools or toolhol...,NaN,EEF,False
2165731,machine tapers,noun,en,en,plural of machine taper,NaN,EEF,False
2165732,authentic assessments,noun,en,en,plural of authentic assessment,NaN,EEF,False
2165733,barn-raisings,noun,en,en,plural of barn-raising,NaN,EEF,False


In [57]:
duplicated_df = one_gloss_defs_df[one_gloss_defs_df['duplicated'] == True]

In [64]:
same_rows = one_gloss_defs_df[one_gloss_defs_df['word'] == one_gloss_defs_df['definition']]

In [65]:
same_rows

,word,pos,lang_code,def_lang,definition,translations,wl_code,duplicated
12,product,noun,nl,en,product,NaN,ENF,False
18,aam,noun,nl,en,aam,NaN,ENF,False
19,aardwolf,noun,nl,en,aardwolf,NaN,ENF,False
26,quiz,noun,nl,en,quiz,NaN,ENF,False
27,millennium,noun,nl,en,millennium,NaN,ENF,False
...,...,...,...,...,...,...,...,...
1349491,K. pneumoniae carbapenemase,noun,en,en,K. pneumoniae carbapenemase,NaN,EEF,False
1385442,RGSS,name,en,en,RGSS,NaN,EEF,False
1513937,I don't want to buy anything,phrase,en,en,I don't want to buy anything,"[{'lang': 'Dutch', 'code': 'nl', 'lang_code': ...",EEF,False
1736129,Litvin,noun,en,en,Litvin,NaN,EEF,False


In [67]:
duplicates = one_gloss_defs_df.groupby('word')['lang_code'].nunique().reset_index()
duplicates = duplicates[duplicates['lang_code'] > 1]

,word,lang_code
4,!Xóõ,2
73,'Auhelawa,2
250,'er,2
339,'m,2
360,'n,2
...,...,...
1920924,ǃKung,2
1921053,β-lactam,2
1921238,◌̀,2
1921239,◌́,2


In [60]:
groups = duplicated_df.groupby(group_keys=['word', 'pos'], by='word')

In [61]:
groups.describe()

KeyboardInterrupt: 